In [1]:
from micrograd.engine import Tensor
import micrograd.nn as nn
import micrograd.functional as F
from micrograd.optim import SGD
from micrograd.data import MnistDataset, DataLoader
import numpy as np
from tqdm import tqdm

CNN definition

In [2]:
class CNN(nn.Module):
    def __init__(self):
        self.conv1 = nn.Conv2d(1, 6, K=5, stride=2, padding=2)
        self.conv2 = nn.Conv2d(6, 16, K=5, stride=2)

        self.lin1 = nn.Linear(400, 120)
        self.lin2 = nn.Linear(120, 80)
        self.lin3 = nn.Linear(80, 10)

    def __call__(self, x):
        x = self.conv1(x).relu()
        x = self.conv2(x).relu()
        x = x.reshape((-1, 400))
        x = self.lin1(x).relu()
        x = self.lin2(x).relu()
        x = self.lin3(x)
        return x

Train

In [3]:
train_dataset = MnistDataset(train=True)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [4]:
model = CNN()
optimizer = SGD(model.parameters(), lr=0.1)

max_epochs = 1
for epoch in range(max_epochs):
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for it, (x, y) in pbar:
        optimizer.zero_grad()
        logits = model(x)
        loss = F.batched_cross_entropy(logits, y)
        loss.backward()
        optimizer.step()
        
        pbar.set_description(f"epoch: {epoch}, it: {it}, loss: {loss.data.item():.5f}")

epoch: 0, it: 1874, loss: 0.11213: 100%|███████████████████| 1875/1875 [01:03<00:00, 29.49it/s]


Test

In [5]:
test_dataset = MnistDataset(train=False)
test_loader = DataLoader(test_dataset, batch_size=128)

In [7]:
correct, total = 0, 0
for x, y in test_loader:
    logits = model(x)
    pred = np.argmax(logits.data, axis=-1)
    correct += (y.data == pred).sum()
    total += x.data.shape[0]

print(f"accuracy: {correct / total}")

accuracy: 0.9642
